<a href="https://colab.research.google.com/github/kevinhhl/options-pricing-tools-and-trading-strategies/blob/BSM-part-2/Black_Scholes_Merton_Model_Part2_Position_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import time
import pandas as pd
from datetime import date
from scipy.stats import norm
!pip install yahoo_fin
from yahoo_fin import options
!pip install yfinance
import yfinance as yf

# Overview

Back in [Part 1](https://github.com/kevinhhl/options-pricing-tools-and-trading-strategies/blob/main/Black_Scholes_Merton_Model_Part1_Screening_YF_for_theoretical_edges.ipynb), we implemented the Black Scholes Model (BSM) and applied it to data provided by Yahoo. We had a summary table to list out the theoretical values and Greeks for each of the individual options in the options chain.

From a portfolio perspective, we need to only include those options that we want to include in our positions. To do so, we need to refactor the code from Part 1, and create a few datatypes of store values in hierarchical forms.



# Implementation

Here is the blueprint for three different levels of datatypes that will be used for organizing options data. Programming it this way will allow us to develop further tools to perform risk analysis for options portfolios from both top-down and bottom-up perspectives.

| Hierarchy & name of datatype | Description |
|--|--|
| Level 1: Contract| A datatype to store data points attributed to an individual option (ie. maturity date, strike price, call/put).  |
| Level 2: Leg | A data structure that stores (1) a single Contract (2) an instance of a BSM, and (3) the output value from BSM|
| Level 3: Position | A collection of multiple Legs |




#### class BSM():

In [ ]:
class BSM:
  
  def __init__(self, x, s,t,r,sigma):
    self.d1, self.d2 = None, None
    self.tv_call, self.delta_call , self.gamma_call, self.vega_call, self.theta_call, self.rho_call = None, None, None, None, None, None
    self.tv_put, self.delta_put, self.gamma_put, self.vega_put, self.theta_put, self.rho_put = None, None, None, None, None, None

    self.x = x
    self.s = s
    self.t = t
    self.r = r
    self.sigma = sigma
    self.calc()

  def calc(self):
    ''' Calculates all the theoretical and Greeks values for call and put. 
        Need to call calc() whenever new input is provided. 
    '''
    _a = math.log(self.s/ self.x)
    _b = (self.r+self.sigma**2/2)*self.t
    self.d1 = (_a+_b)/self.sigma*math.sqrt(self.t)
    self.d2 = self.d1 - self.sigma * math.sqrt(self.t)
    
    # Call: 
    self.tv_call    = self.s * norm.cdf(self.d1) - self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2)
    self.delta_call = norm.cdf(self.d1)
    self.gamma_call = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_call  = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_call = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) - self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    self.rho_call   = 0.01*(self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(self.d2))
    
    # Put: 
    self.tv_put    = self.x * math.exp(-self.r*self.t)-self.s+self.tv_call
    self.delta_put = -norm.cdf(-self.d1)
    self.gamma_put = norm.pdf(self.d1)/(self.s*self.sigma*math.sqrt(self.t))
    self.vega_put  = 0.01*(self.s*norm.pdf(self.d1)*math.sqrt(self.t))
    self.theta_put = 0.01*(-(self.s*norm.pdf(self.d1)*self.sigma)/(2*math.sqrt(self.t)) + self.r*self.x*math.exp(-self.r*self.t)*norm.cdf(-self.d2))
    self.rho_put   = 0.01*(-self.x*self.t*math.exp(-self.r*self.t)*norm.cdf(-self.d2))



#### class contract():

In [ ]:
class contract:

  def __init__(self, expiry_date, strike, contracttype, quoted_price):
    self.multiple = 100
    self.strike = strike
    self.contracttype=contracttype
    self.expiry_date = expiry_date
    self.price = quoted_price

  def __str__(self):
    return "{}_{}_{} @{}".format(self.expiry_date, self.strike, self.contracttype, self.price)

#### class leg():

In [ ]:
class leg:

  def __init__(self, side, contract, size, date_today, s,r,sigma):
    self.size=size
    leg.size = size
    self.side = side
    t = (contract.expiry_date-date_today).days/365
    self.contract = contract
    self.model = BSM(contract.strike, s,t,r,sigma)

    self.delta = None
    self.rho = None
    self.vega = None
    self.theta = None   
    self.gamma = None
    self.edge = 0
    self.premium = 0

    # int _coefficient, has three purposes:
    #   (1) +/-ve Delta for long call/put, but it's reversed from seller's perspective  
    #   (2) If buyer of contract:   +ve vega, -ve theta, +ve gamma
    #   (3) If seller of contract:  -ve vega, +ve theta, -ve gamma
    _coefficient = None
    if self.side == "long":
      _coefficient = 1
    elif self.side == "short":
      _coefficient = -1
    
    if contract.contracttype=="call":
      if self.side == "long":
        self.edge = (self.model.tv_call-self.contract.price) * size 
      elif self.side == "short":
        self.edge = (self.contract.price-self.model.tv_call) * size 
      self.delta = self.model.delta_call * size  * _coefficient
      self.rho = self.model.rho_call * size      * _coefficient
      self.vega = self.model.vega_call * size    * _coefficient
      self.gamma = self.model.gamma_call * size  * _coefficient
      self.theta = self.model.theta_call * size  * _coefficient
    
    elif contract.contracttype=="put":
      if self.side == "long":
        self.edge = (self.model.tv_put-self.contract.price) * size 
      elif self.side == "short":
        self.edge = (self.contract.price-self.model.tv_put) * size 
      self.delta = self.model.delta_put * size   * _coefficient
      self.rho = self.model.rho_put * size       * _coefficient
      self.vega = self.model.vega_put * size     * _coefficient
      self.gamma = self.model.gamma_put * size   * _coefficient
      self.theta = self.model.theta_put * size   * _coefficient
    
    self.premium = contract.price * size * -_coefficient

  def __str__(self):
    n_round= 4 
    return "{} [x{}]: Δ={},𝚪={},Θ={},V={},⍴={}".format(str(self.contract),self.size, \
      self.delta.round(n_round),self.gamma.round(n_round),self.theta.round(n_round),self.vega.round(n_round),self.rho.round(n_round))


class position:

In [ ]:
class position:

  def __init__(self):
    self.list_leg = []
    self.total_delta = 0
    self.total_gamma = 0
    self.total_vega = 0
    self.total_theta = 0
    self.total_rho = 0
    self.total_premium = 0
    self.total_th_edge = 0

  def get_summary(self, rounding=4) -> dict:
    ''' Returns dict of {string:float}
    '''
    return {"Total delta" : round(self.total_delta,rounding),
            "Total gamma" : round(self.total_gamma,rounding),
            "Total vega"  : round(self.total_vega,rounding),
            "Total theta" : round(self.total_theta,rounding),
            "Total rho"   :round(self.total_rho,rounding),
            "Premium (paid)/received" : round(self.total_premium,rounding),
            "Th. Edge"    : round(self.total_th_edge,rounding),
            }
  
  def add_leg(self, leg):
    ''' @param leg
    '''
    size = leg.size
    self.total_premium += leg.premium
    self.total_th_edge += leg.edge
    self.list_leg.append(leg)
    
    self.total_delta += leg.delta
    self.total_gamma += leg.gamma
    self.total_vega += leg.vega
    self.total_theta += leg.theta
    self.total_rho += leg.rho


#Application:
😎 Setting up our model:

In [ ]:
# Manual inputs:
symbol                   = "TSLA" 
sigma                    =0.70 
riskfree_rate            = .0512 
date_today               = date(2023,2,18) 
date_expire              = date(2023,2,24) 

In [ ]:
ttm = (date_expire-date_today).days/365

# Confirming that the expiration date is valid.
exp_dates = options.get_expiration_dates(symbol)
date_expire_str = date_expire.strftime("%B %d, %Y") 
assert date_expire_str in exp_dates 

# Obtaining the recent close priceticker_yahoo = yf.Ticker(symbol)
ticker_yahoo = yf.Ticker(symbol)
data = ticker_yahoo.history()
crnt_price = data['Close'].iloc[-1]
crnt_price

197.3699951171875

---
##Option Chain Analysis:

In [ ]:
chain_call = options.get_options_chain(symbol)["calls"]
chain_put = options.get_options_chain(symbol)["puts"]

In [ ]:
def populate_dict(yf_chain, contracttype):
  ''' @param Object yf_chain : from options.get_options_chain(symbol, date_expire_str)["calls" or "puts"]
      @param string contracttype : either "call" or "put
      Returns a dict of {float strike price : contract object}
  '''
  out = {}
  for i in range(len(yf_chain)):
    x = yf_chain["Strike"][i]
    out[x] = contract(date_expire, x, contracttype, yf_chain["Last Price"][i])
  return out

calls = populate_dict(chain_call, "call")
puts = populate_dict(chain_put, "put")


## Position Analysis

**Long straddle:**

In [ ]:
pos = position()
leg1 = leg(side="long", contract=calls[207.5], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
leg2 = leg(side="long", contract=puts[207.5], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
pos.add_leg(leg1)
pos.add_leg(leg2)
for e in pos.list_leg:
  print(e)
pos.get_summary()

2023-02-24_207.5_call @2.37 [x1]: Δ=0.4967,𝚪=0.0225,Θ=-2.1983,V=0.1009,⍴=0.0157
2023-02-24_207.5_put @12.45 [x1]: Δ=-0.5033,𝚪=0.0225,Θ=-2.0922,V=0.1009,⍴=-0.0184


{'Total delta': -0.0066,
 'Total gamma': 0.045,
 'Total vega': 0.2019,
 'Total theta': -4.2905,
 'Total rho': -0.0027,
 'Premium (paid)/received': -14.82,
 'Th. Edge': 0.0662}

**Short straddle:**

In [ ]:
pos = position()
leg1 = leg(side="short", contract=calls[207.5], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
leg2 = leg(side="short", contract=puts[207.5], size=1, date_today=date_today, s=crnt_price, r=riskfree_rate, sigma=sigma)
pos.add_leg(leg1)
pos.add_leg(leg2)
for e in pos.list_leg:
  print(e)
pos.get_summary()

2023-02-24_207.5_call @2.37 [x1]: Δ=-0.4967,𝚪=-0.0225,Θ=2.1983,V=-0.1009,⍴=-0.0157
2023-02-24_207.5_put @12.45 [x1]: Δ=0.5033,𝚪=-0.0225,Θ=2.0922,V=-0.1009,⍴=0.0184


{'Total delta': 0.0066,
 'Total gamma': -0.045,
 'Total vega': -0.2019,
 'Total theta': 4.2905,
 'Total rho': 0.0027,
 'Premium (paid)/received': 14.82,
 'Th. Edge': -0.0662}

### Let's validate the results.

In theory, +ve and -ve signs should be: 

> *Summary of straddle spreads [1]:*
> 
>|Spread|Delta|Gamma|Theta|Vega|Downside|Upside|
|-|-|-|-|-|-|-|
|Long straddle|0|+ve|-ve|+ve|Unlimited reward|Unlimited reward|
|Short straddle|0|-ve|+ve|-ve|Unlimited risk|Unlimited risk|

In a perfect world, deltas for ATM calls and puts are exactly 0.5 and -0.5, respectively. We assume our spreads to be delta neutral at the very beginning. Of course in practice this will not be the case. We will be satisfied if ATM Deltas are approximately close to 0.5.

<br>

---
*References:*

[1] Natenberg, Sheldon. <i>Chapter 11, Option Volatility and Pricing, Second Edition</i>. McGraw-Hill Edu., 2015.